In [1]:
# model and X_test, y_test, etc. are defined by this script
from credoai.datasets import fetch_credit_model

# model and data are defined by this script
X_test, y_test, sensitive_features_test, model = fetch_credit_model()

/home/niwinz/.pyenv/versions/3.10.11/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [5]:
sensitive_features_test


2308     female
22404      male
23397      male
25058    female
2664       male
          ...  
3211       male
9355       male
28201      male
19705      male
28313      male
Name: SEX, Length: 7500, dtype: category
Categories (2, object): ['female', 'male']

In [15]:
y_test

2308     0
22404    0
23397    0
25058    0
2664     1
        ..
3211     0
9355     0
28201    1
19705    1
28313    0
Name: target, Length: 7500, dtype: int64

In [9]:
from credoai.lens import Lens
from credoai.artifacts import ClassificationModel, TabularData
from credoai.evaluators import ModelFairness, Performance

In [12]:
credo_model = ClassificationModel(name="credit_default_classifier", model_like=model)
credo_data = TabularData(
    name="UCI-credit-default",
    X=X_test,
    y=y_test,
    sensitive_features=sensitive_features_test,
)
lens = Lens(model=credo_model, assessment_data=credo_data)
metrics = ['precision_score', 'recall_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics))
lens.add(Performance(metrics=metrics))
lens.run()

2023-04-12 18:45:15,483 - lens - INFO - Evaluator ModelFairness added to pipeline. Sensitive feature: SEX
2023-04-12 18:45:15,750 - lens - INFO - Evaluator Performance added to pipeline. 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished


In [16]:
results = lens.get_results()
print(f"Results for {len(results)} evaluators")

Results for 2 evaluators


In [17]:
performance_results = lens.get_results(evaluator_name='Performance')[0]
# first dataframe is overall metrics
display(performance_results['results'][0])
# second dataframe is the long form of the confusion matrix
display(performance_results['results'][1])

,type,value
0,precision_score,0.628081
1,recall_score,0.360172


,true_label,predicted_label,value
0,0,0,0.940916
1,1,0,0.639828
2,0,1,0.059084
3,1,1,0.360172
